# Helper functions for Tensorflow Fold



In [ ]:
length = td.Length()

embedded =  (td.InputTransform(lambda s: [index(x) for x in s]) >> 
             td.Map(td.Scalar(tf.int32) >> 
             td.Function(td.Embedding(vsize, EMBEDDING_SIZE))))

onehot = (td.InputTransform(lambda s: [index(x) for x in s]) >>
          td.Map(td.Scalar(tf.int32) >>
          td.Function(lambda indices: tf.one_hot(indices, depth=vsize)) >>
          td.Function(lambda x: tf.reshape(x, [-1,vsize,1]))))

decode_onehot = td.InputTransform(lambda s: [char(np.argmax(np.squeeze(x))) for x in s])

print(decode_onehot.eval(onehot.eval("malú")))
#embedded.eval("kacsa")

In [ ]:
def conv1d_on_sequence(x, scope, kernel_size=3, input_channels=72, output_channels=72):
    with tf.variable_scope(scope) as sc:
        filters = tf.get_variable("conv_filter", [kernel_size] +  [input_channels, output_channels] , initializer=tf.truncated_normal_initializer(stddev=0.1))
        bias = tf.get_variable("conv_bias",  output_channels, initializer=tf.constant_initializer(0.05, dtype=tf.float32))
        conv = tf.nn.conv1d(x, filters=filters, stride=1, padding='VALID')
        return tf.nn.relu(tf.add(conv, bias))
    
def SeqToTuple(T, N):
    return (td.InputTransform(lambda x: tuple(x))
            .set_input_type(td.SequenceType(T))
            .set_output_type(td.Tuple(*([T] * N))))   

In [ ]:
def bidirectional_conv_LSTM():
    convlstm = Conv1DLSTMCell(input_shape=[vsize,1], output_channels=8, kernel_shape=[5])
    conv_lstm_cell_1d = td.ScopedLayer(convlstm)

    bidir_conv_lstm = td.Composition()
    with bidir_conv_lstm.scope():
        data = td.Record((td.Map(
                                td.Vector(vsize) >>
                                td.Function(lambda x: tf.reshape(x, [-1,vsize,1]))),
                          td.Map(
                                td.Vector(vsize) >>
                                td.Function(lambda x: tf.reshape(x, [-1,vsize,1]))))).reads(bidir_conv_lstm.input)

        forward = td.Identity().reads(data[0])
        backward = td.Identity().reads(data[1])

        forw = (td.RNN(conv_lstm_cell_1d) >>
                td.GetItem(1) >>
                td.GetItem(0) >>
                td.Function(lambda rnn_outs: tf.contrib.layers.flatten(rnn_outs))).reads(forward)

        backw = (td.RNN(conv_lstm_cell_1d) >>
                 td.GetItem(1) >>
                 td.GetItem(0) >>
                 td.Function(lambda rnn_outs: tf.contrib.layers.flatten(rnn_outs))).reads(backward)

        rnn_outs = td.Concat().reads(forw,backw)
        bidir_conv_lstm.output.reads(rnn_outs)
    return bidir_conv_lstm >> td.FC(1)


def FCNN():
    return td.FC(50) >> td.FC(1)# >> td.Function(lambda xs: tf.squeeze(xs, axis=1))

#bidir = bidirectional_conv_LSTM()
#fc = FCNN()
#blk = bidir# >> fc 
#blk.eval([a,a])

In [ ]:
cell = td.ScopedLayer(tf.contrib.rnn.BasicLSTMCell(num_units=16), 'char_cell')
blk = td.Map(td.Vector(vsize) >> td.Function(lambda x: tf.reshape(x, [-1,vsize]))) >> td.RNN(cell) >> td.GetItem(1) >> td.GetItem(0) >> td.FC(1)#>> td.Function(lambda xs: tf.squeeze(xs, axis=1))